In [1]:
import json
import os
import numpy as np
import csv
import easydict
import cv2

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.utils import save_image
from torch.autograd import Variable

from PIL import Image

from tqdm import tqdm

from time import sleep

In [2]:
from torchvision import utils
from torchvision import datasets
import itertools
import matplotlib.pyplot as plt

import random
random.seed(100)

In [3]:
mnist_train = datasets.MNIST(root='./MNIST_data/', # 다운로드 경로 지정
                          train=True, # True를 지정하면 훈련 데이터로 다운로드
                          transform=transforms.ToTensor(), # 텐서로 변환
                          download=True)

9913344it [00:00, 20062832.52it/s]                            


Extracting ./MNIST_data/MNIST\raw\train-images-idx3-ubyte.gz to ./MNIST_data/MNIST\raw



29696it [00:00, 1488598.95it/s]          


Extracting ./MNIST_data/MNIST\raw\train-labels-idx1-ubyte.gz to ./MNIST_data/MNIST\raw



1649664it [00:00, 1700855.05it/s]                             


Extracting ./MNIST_data/MNIST\raw\t10k-images-idx3-ubyte.gz to ./MNIST_data/MNIST\raw



5120it [00:00, 5136291.91it/s]          

Extracting ./MNIST_data/MNIST\raw\t10k-labels-idx1-ubyte.gz to ./MNIST_data/MNIST\raw




C:\Users\akzns\anaconda3\lib\site-packages\torchvision\datasets\mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


## DataLoader

In [4]:
train_dataloader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=1,
                                          shuffle=True,
                                          drop_last=True)

In [5]:
total_batch = len(train_dataloader)
print(total_batch)
a,b=next(iter(train_dataloader))
print(a.shape, b.shape)

60000
torch.Size([1, 1, 28, 28]) torch.Size([1])


In [6]:
train_data = list()
train_label = list()

for i, data in enumerate(train_dataloader):
    imgs, labels = data
    train_data.append(np.asarray(imgs.squeeze().view(28,28,1)))
    train_label.append(int(labels))

In [7]:
num_img_per_class = 3000
class_list=datasets.MNIST.classes
print(a)

tensor([[[[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 

In [8]:
a.size()

torch.Size([1, 1, 28, 28])

In [9]:
class My_MNIST_Dataset(Dataset):

    def __init__(self, data, labels, transform=None):
        """
        Args:
            csv_file (string): csv file path
            root_dir (string): Directory path where all images exist
            transform (callable, optional): Optical transform to be applied to the sample
        """
        self.data = data
        self.labels = labels
        # MNIST dataset class count
        self.classnum = 10
        self.imsize = parser['img_size']

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # data resize and one-hot encoding
        data_torch = torch.from_numpy(cv2.resize(self.data[idx],(self.imsize,self.imsize)))
        data_torch2vec = data_torch.view(1*self.imsize*self.imsize)

        # core
        label_torch = torch.from_numpy(np.zeros(self.classnum))
        label_torch[self.labels[idx]] = 1
        return data_torch2vec, label_torch

In [10]:
parser = easydict.EasyDict({"n_epochs": 200,
                            "batch_size":100, 
                            "lr":0.0002, 
                            "b1":0.5, 
                            "b2":0.999, 
                            "n_cpu":8, 
                            "latent_dim":100, 
                            "n_classes":len(class_list), 
                            "img_size":28, 
                            "channels":3, 
                            "sample_interval":1})

In [11]:
cuda = True if torch.cuda.is_available() else False
img_shape = (parser.channels, parser.img_size, parser.img_size)

In [12]:
img_shape

(3, 28, 28)

## Generator part

In [13]:
# model class
class Generator(torch.nn.Module):
  def __init__(self):
    super(Generator, self).__init__()
    size = parser['img_size']

    # z = noise, y = class for consistency
    self.layer_z = torch.nn.Linear(100, 200)
    self.relu_z = torch.nn.ReLU()

    self.layer_y = torch.nn.Linear(parser.n_classes, 1000)
    self.relu_y = torch.nn.ReLU()

    self.layer_out = torch.nn.Linear(1200, 1*size*size)

  def forward(self, z, y):
    z_hidden = self.relu_z(self.layer_z(z))
    y_hidden = self.relu_y(self.layer_y(y))
    

    concat_hidden = torch.cat([z_hidden,y_hidden],dim=1)
    output_vec = self.layer_out(concat_hidden)
    output = torch.sigmoid(output_vec)
    return output

## Discriminator part

In [16]:
# model class
class Discriminator(torch.nn.Module):
  def __init__(self):
    super(Discriminator, self).__init__()
    size = parser['img_size']

    self.layer_x = torch.nn.Linear(1*size*size, 1200)
    self.relu_x = torch.nn.ReLU()

    self.layer_y = torch.nn.Linear(parser.n_classes, 250)
    self.relu_y = torch.nn.ReLU()

    self.layer_out = torch.nn.Linear(1450, 960)
    self.relu_out = torch.nn.ReLU()

    self.layer_score = torch.nn.Linear(960, 1)

  def forward(self, x, y):
    x_hidden = self.relu_x(self.layer_x(x))
    y_hidden = self.relu_y(self.layer_y(y))
    concat_hidden = torch.cat([x_hidden, y_hidden], dim=1)
# TO DO (4) starts here
    output_vec = self.relu_out(self.layer_out(concat_hidden))#Fill Here#
    output_score = self.layer_score(output_vec)#Fill Here#
    output = torch.sigmoid(output_score)#Fill Here#
# TO DO (4) ends here
    return output

## Sample image checking

In [17]:
def sample_image(z, n_row, epoch):
    size = parser['img_size']
    # Sample noise
    gen_labels = []
    # Get labels ranging from 0 to n_classes for n rows
    for randpos in range(10):
      gen_labels.append(torch.eye(parser.n_classes)[randpos])
    gen_labels = torch.stack(gen_labels).cuda()
    gen_imgs = generator(z, gen_labels)
    save_image(gen_imgs.view(n_row,1,size,size).data, "./assignment4_arrange_images/assignment4_arrange_images4/%s.png" % str(epoch).zfill(3), nrow=n_row, normalize=True)

In [19]:
# Main
adversarial_loss = torch.nn.MSELoss().cuda()
generator = Generator().cuda()
discriminator = Discriminator().cuda()

mnist_train_dataset = My_MNIST_Dataset(train_data, train_label, parser['img_size'])                   

mnist_train_dataloader = DataLoader(mnist_train_dataset, batch_size=parser.batch_size, shuffle=True)

optimizer_G = torch.optim.Adam(generator.parameters(), lr=0.0002)
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=0.0002)

In [20]:
# Directory 생성
if not os.path.exists('./assignment4_arrange_images/assignment4_arrange_images4'):
  os.makedirs('./assignment4_arrange_images/assignment4_arrange_images4')

z_test = torch.randn(10, parser.latent_dim).type(torch.FloatTensor).cuda()

# Train
generator.train()
discriminator.train()

g_loss = torch.Tensor([0])
d_loss = torch.Tensor([0])

for epoch in range(parser.n_epochs):
  for batch_idx, (img_torch2vec, label_torch) in enumerate(mnist_train_dataloader):
    img_torch2vec = img_torch2vec.type(torch.FloatTensor).cuda()  #to GPU
    label_torch = label_torch.type(torch.FloatTensor).cuda()

    # Adversarial ground truths
    valid = torch.ones(parser.batch_size, 1).cuda()
    fake = torch.zeros(parser.batch_size, 1).cuda()

    # Configure input
    real_imgs = img_torch2vec
    labels = label_torch

    # Train Gen
    optimizer_G.zero_grad()

    # Sample noise and labels as generator input
    z = torch.randn(parser.batch_size, parser.latent_dim).cuda()
    gen_labels = []
    for randpos in np.random.randint(0, parser.n_classes, parser.batch_size):
      gen_labels.append(torch.eye(parser.n_classes)[randpos])
    gen_labels = torch.stack(gen_labels).cuda()

    # Generate a batch of images
    gen_imgs = generator(z, gen_labels)
    
    # Loss measures generator's ability to fool the discriminator
    val_output = discriminator(gen_imgs, gen_labels)
    # g_loss = adversarial_loss(val_output, valid)
    g_loss = -torch.mean(torch.log(val_output + 1e-8))

    g_loss.backward()
    optimizer_G.step()

    if batch_idx % 4 == 0:
      # Train Disc
      optimizer_D.zero_grad()
      
      validity_real = discriminator(real_imgs, labels)
      d_real_loss = adversarial_loss(validity_real, valid)

      # val = output         
      validity_fake = discriminator(gen_imgs.detach(), gen_labels)
      d_fake_loss = adversarial_loss(validity_fake, fake)

      # d_loss = (d_real_loss + d_fake_loss) / 2
      d_loss = -torch.mean(torch.log(validity_real + 1e-8) + torch.log(1 - validity_fake + 1e-8))

      d_loss.backward()
      optimizer_D.step()
    if batch_idx % 100 == 0:
      print('{:<13s}{:<8s}{:<6s}{:<10s}{:<8s}{:<9.5f}{:<8s}{:<9.5f}'.format('Train Epoch: ', '[' + str(epoch) + '/' + str(parser['n_epochs']) + ']', 'Step: ', '[' + str(batch_idx) + '/' + str(len(mnist_train_dataloader)) + ']', 'G loss: ', g_loss.item(), 'D loss: ', d_loss.item()))

  if epoch % parser.sample_interval == 0:
    sample_image(z_test, n_row=10, epoch=epoch)

Train Epoch: [0/200] Step: [0/600]   G loss: 0.68158  D loss: 1.38345  
Train Epoch: [0/200] Step: [100/600] G loss: 2.01697  D loss: 0.38278  
Train Epoch: [0/200] Step: [200/600] G loss: 1.56921  D loss: 0.41538  
Train Epoch: [0/200] Step: [300/600] G loss: 1.53077  D loss: 0.58407  
Train Epoch: [0/200] Step: [400/600] G loss: 2.54342  D loss: 0.67567  
Train Epoch: [0/200] Step: [500/600] G loss: 2.95479  D loss: 0.29972  
Train Epoch: [1/200] Step: [0/600]   G loss: 2.55594  D loss: 0.32471  
Train Epoch: [1/200] Step: [100/600] G loss: 3.45636  D loss: 0.10174  
Train Epoch: [1/200] Step: [200/600] G loss: 4.70273  D loss: 0.08725  
Train Epoch: [1/200] Step: [300/600] G loss: 3.11912  D loss: 0.34770  
Train Epoch: [1/200] Step: [400/600] G loss: 3.68498  D loss: 0.24417  
Train Epoch: [1/200] Step: [500/600] G loss: 3.88842  D loss: 0.08626  
Train Epoch: [2/200] Step: [0/600]   G loss: 4.01800  D loss: 0.04227  
Train Epoch: [2/200] Step: [100/600] G loss: 4.04896  D loss: 0.